For Part A Task1,

we have chosen embedded format for our data model.

- the example of the data model.

{
 date_as_id: 31-12-2017
 station:  948700,
 air_temperature_celcius: 19 ,
 relative_humidity: 56.8,
 windspeed_knots: 7.9,	
 max_wind_speed: 11.1,
 precipitation:0.00I,
 hotspot_histories:[
            {					
 		    latitude:-37.966,
 		    longitude:145.051,
		    datetime:2018-12-27T04:16:51,
		    confidence:78,
         	surface_temperature_celcius:68
        	},

		    {					
		    latitude:-35.541,
		    longitude:143.311,
		    datetime:2018-12-27T00:02:15,
		    confidence:82,
		    surface_temperature_celcius:63
		    }
                   ]
}


- The justification for choosing that data model 
-Embedded format-read two datasets and merge them to single doc
-Referencing format- save them as two different collections removing duplicates.

Embedded format has easier query format but tend to take more spaces.
Referencing may be able to save more spaces as compared to Embedded format.

However, the amount of duplicates between climate_hisotry.csv and hotspot_history.csv is actually few such as only date attribute.
I assume that Embedded format could be better because it has better query structure and referencing model in this case would not really save memories.

In [1]:
import pymongo
from pymongo import MongoClient

# Method 1: connect on the default host and port
client = MongoClient () # method 1: connect on the default host and port

In [2]:

db= client.fit3182_assignment_db

#collection = db.climate_historic

from pprint import pprint

cursor = db.list_collections()
for collection in cursor:
    pprint(collection)

{'idIndex': {'key': {'_id': 1},
             'name': '_id_',
             'ns': 'fit3182_assignment_db.test',
             'v': 2},
 'info': {'readOnly': False,
          'uuid': UUID('2bb4ca6e-c804-4d91-830f-1e503c5552d8')},
 'name': 'test',
 'options': {},
 'type': 'collection'}
{'idIndex': {'key': {'_id': 1},
             'name': '_id_',
             'ns': 'fit3182_assignment_db.climate_historic',
             'v': 2},
 'info': {'readOnly': False,
          'uuid': UUID('8e93b5f2-8d41-470a-aea9-94f03896c836')},
 'name': 'climate_historic',
 'options': {},
 'type': 'collection'}
{'idIndex': {'key': {'_id': 1},
             'name': '_id_',
             'ns': 'fit3182_assignment_db.Mycollection',
             'v': 2},
 'info': {'readOnly': False,
          'uuid': UUID('f9ca9ee0-7edd-4364-83bd-9a9842047061')},
 'name': 'Mycollection',
 'options': {},
 'type': 'collection'}
{'idIndex': {'key': {'_id': 1},
             'name': '_id_',
             'ns': 'fit3182_assignment_db.Testcollect


-Read csv files and merge them.


In [4]:
# Import pandas 
import pandas as pd 
import json
  
# reading csv file  
climate = pd.read_csv("/home/student/Documents/climate_historic.csv")#set Your directry to access csv file
hotspot = pd.read_csv("/home/student/Documents/hotspot_historic.csv")#set your directry to access csv file

print(climate["station"][1],climate["date"][1],climate["air_temperature_celcius"][1],climate["relative_humidity"][1],
climate["windspeed_knots"][1],climate["max_wind_speed"][1],climate["precipitation "][1])

print(len(climate))

print(hotspot.head(1))
print(hotspot["latitude"][1],hotspot["longitude"][1],hotspot["datetime"][1],hotspot["confidence"][1],hotspot["date"][1],hotspot["surface_temperature_celcius"][1])

# The first step in the merge-all groupby method
def groupby(dataset1,dataset2):
    """
    Perform a groupby method

    Arguments:
    dataset -- two tables

    Return:
    result -- the grouped record by its date
    """
    
    
    dict = {}
    ##Create key for each record in climate
    for i in range(len(dataset1)):
        key = dataset1["date"][i]
        if key not in dict:
            dict[key] =[i,dataset1["date"][i],[]]
    
    ##Printing 5 items just to check if key is correct or not 
    for i in range(5):
        key=dataset1["date"][i]
        print(dict[key])
    
    for i in range(len(dataset2)):
        key = dataset2["date"][i]
        if key not in dict:
            print("key not found")
        dict[key][2].append(i) ##append index of record
    
    return dict
        

result = groupby(climate,hotspot)




    
    

948700 2/01/2018 15 50.7 9.2 13.0  0.02G
366
   latitude  longitude             datetime  confidence        date  \
0   -37.966    145.051  2018-12-27T04:16:51          78  27/12/2018   

   surface_temperature_celcius  
0                           68  
-35.541 143.311 2018-12-27T00:02:15 82 27/12/2018 63
[0, '31/12/2017', []]
[1, '2/01/2018', []]
[2, '3/01/2018', []]
[3, '4/01/2018', []]
[4, '5/01/2018', []]



-Speciy the json format (Embedded format is chosen)
-Save it as collection named "Mycollection"


In [5]:
import datetime as dt
Mycollection = db.Mycollection

for i in range(len(climate)):
    key = climate["date"][i]
    item = result[key] ##[index,date,array of indices of mathced records]
    jsonData = {}
    jsonData["station"] = int(climate["station"][i])
    jsonData["date"] = str(dt.datetime.strptime(climate["date"][i], "%d/%m/%Y").date())
    jsonData["air_temperature_celcius"] = int(climate["air_temperature_celcius"][i])
    jsonData["relative_humidity"] = float(climate["relative_humidity"][i])
    jsonData["windspeed_knots"] = float(climate["windspeed_knots"][i])
    jsonData["max_wind_speed"] = float(climate["max_wind_speed"][i])
    jsonData["precipitation "] = climate["precipitation "][i]
    ##for hotspot
    #jsonData["hotspot_histories"] = []
    array = []
    for index in item[2]:
        jsonData2 = {}
        jsonData2["latitude"] = float(hotspot["latitude"][index])
        jsonData2["longitude"] = float(hotspot["longitude"][index])
        jsonData2["datetime"] = hotspot["datetime"][index]
        jsonData2["confidence"] = int(hotspot["confidence"][index])
        jsonData2["surface_temperature_celcius"] = int(hotspot["surface_temperature_celcius"][index])
        
        array.append(jsonData2)
    #print(array)
    jsonData["hotspot_histories"] = array
    #pprint(jsonData)
    Mycollection.insert_one(jsonData)



In [3]:
#db.Mycollection.drop()

In [6]:
##To check if the record is inserted correctly

collection = db.Mycollection
#print(collection.find().count())
cursor = collection.find()
for i in range(100): 
    pprint(cursor[i])


{'_id': ObjectId('5ede169566e94f863c8e1031'),
 'air_temperature_celcius': 19,
 'date': '2017-12-31',
 'hotspot_histories': [],
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'_id': ObjectId('5ede169566e94f863c8e1032'),
 'air_temperature_celcius': 15,
 'date': '2018-01-02',
 'hotspot_histories': [],
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'_id': ObjectId('5ede169566e94f863c8e1033'),
 'air_temperature_celcius': 16,
 'date': '2018-01-03',
 'hotspot_histories': [],
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'_id': ObjectId('5ede169566e94f863c8e1034'),
 'air_temperature_celcius': 24,
 'date': '2018-01-04',
 'hotspot_histories': [],
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 61.6,
 'station': 948700,
 'windspe

                        'latitude': -37.9257,
                        'longitude': 143.4891,
                        'surface_temperature_celcius': 54},
                       {'confidence': 64,
                        'datetime': '2018-03-19T04:36:00',
                        'latitude': -37.3982,
                        'longitude': 147.0001,
                        'surface_temperature_celcius': 58},
                       {'confidence': 90,
                        'datetime': '2018-03-19T04:36:00',
                        'latitude': -37.4523,
                        'longitude': 147.0175,
                        'surface_temperature_celcius': 66},
                       {'confidence': 59,
                        'datetime': '2018-03-19T04:35:00',
                        'latitude': -37.4023,
                        'longitude': 147.0394,
                        'surface_temperature_celcius': 56},
                       {'confidence': 60,
                        'datetime': '2018-0

 'hotspot_histories': [{'confidence': 76,
                        'datetime': '2018-04-04T15:33:40',
                        'latitude': -37.2162,
                        'longitude': 146.2408,
                        'surface_temperature_celcius': 38},
                       {'confidence': 93,
                        'datetime': '2018-04-04T15:31:00',
                        'latitude': -36.086999999999996,
                        'longitude': 141.7227,
                        'surface_temperature_celcius': 44},
                       {'confidence': 63,
                        'datetime': '2018-04-04T15:30:40',
                        'latitude': -36.0933,
                        'longitude': 141.7154,
                        'surface_temperature_celcius': 35},
                       {'confidence': 100,
                        'datetime': '2018-04-04T15:30:40',
                        'latitude': -36.0852,
                        'longitude': 141.7112,
                        'surface

                        'latitude': -38.3327,
                        'longitude': 143.8759,
                        'surface_temperature_celcius': 50},
                       {'confidence': 77,
                        'datetime': '2018-04-06T04:20:40',
                        'latitude': -37.8407,
                        'longitude': 147.1609,
                        'surface_temperature_celcius': 51},
                       {'confidence': 75,
                        'datetime': '2018-04-06T04:20:40',
                        'latitude': -35.9065,
                        'longitude': 143.6763,
                        'surface_temperature_celcius': 49},
                       {'confidence': 87,
                        'datetime': '2018-04-06T04:20:40',
                        'latitude': -36.4339,
                        'longitude': 141.5458,
                        'surface_temperature_celcius': 62},
                       {'confidence': 76,
                        'datetime': '2018-0

the merging step is done.
Now we can start query step


a. Find climate data on 10th December 2018.  

b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C. 

c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2018.  

d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.  

e. Find the top 10 records with the highest surface temperature (°C). 

f. Find the number of fire in each day. You are required to only display the total number of fire and the date in the output. 

g. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [8]:
print("query a ")
pprint(collection.find_one({"date":"2018-12-10"}))


query a 
{'_id': ObjectId('5ede169566e94f863c8e1188'),
 'air_temperature_celcius': 17,
 'date': '2018-12-10',
 'hotspot_histories': [{'confidence': 50,
                        'datetime': '2018-12-10T00:57:59',
                        'latitude': -37.986999999999995,
                        'longitude': 144.005,
                        'surface_temperature_celcius': 38},
                       {'confidence': 67,
                        'datetime': '2018-12-10T00:56:57',
                        'latitude': -34.289,
                        'longitude': 141.71200000000002,
                        'surface_temperature_celcius': 54}],
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 53.5,
 'station': 948702,
 'windspeed_knots': 7.3}


In [7]:
print("query b")
#b. Find the latitude, longitude, surface temperature (°C), and confidence 
#when the surface temperature (°C) was between 65 °C and 100 °C

results = collection.aggregate([
  {'$unwind': "$hotspot_histories"}, 
  {'$match': { '$and':[ {"hotspot_histories.surface_temperature_celcius": {"$gte":65}},
                       {"hotspot_histories.surface_temperature_celcius": {"$lte":100}}  ] }} ,
  {
    "$project": {  
          "_id":0,            
          #"datetime" : "$hotspot_histories.datetime",
          "latitude" : "$hotspot_histories.latitude",
          "longitude" : "$hotspot_histories.longitude",
          "surface temperature celcius" : "$hotspot_histories.surface_temperature_celcius",
          "confidence" : "$hotspot_histories.confidence"
      }
  }
  #{'$group': { "_id": '$hotspot_histories.datetime','celcius':"$hotspot_histories.surface_temperature_celcius" }}
  #{'$group': { "_id": '$hotspot_histories.datetime','celcius': { '$push': "$hotspot_histories.surface_temperature_celcius" }}}
])


for result in results:
    #result=result.find({},{'surface_temperature_celcius':1})
    pprint(result)







query b
{'confidence': 94,
 'latitude': -37.2284,
 'longitude': 147.9187,
 'surface temperature celcius': 73}
{'confidence': 97,
 'latitude': -37.6572,
 'longitude': 142.0703,
 'surface temperature celcius': 80}
{'confidence': 84,
 'latitude': -37.0193,
 'longitude': 148.1459,
 'surface temperature celcius': 71}
{'confidence': 100,
 'latitude': -37.4229,
 'longitude': 147.02700000000002,
 'surface temperature celcius': 99}
{'confidence': 80,
 'latitude': -37.0055,
 'longitude': 148.1582,
 'surface temperature celcius': 68}
{'confidence': 85,
 'latitude': -37.4128,
 'longitude': 147.0242,
 'surface temperature celcius': 98}
{'confidence': 90,
 'latitude': -34.357,
 'longitude': 141.5361,
 'surface temperature celcius': 67}
{'confidence': 93,
 'latitude': -34.3539,
 'longitude': 141.5547,
 'surface temperature celcius': 72}
{'confidence': 90,
 'latitude': -36.9939,
 'longitude': 148.2244,
 'surface temperature celcius': 68}
{'confidence': 95,
 'latitude': -36.9959,
 'longitude': 148.2118

 'surface temperature celcius': 72}
{'confidence': 83,
 'latitude': -36.3195,
 'longitude': 146.0601,
 'surface temperature celcius': 77}
{'confidence': 100,
 'latitude': -36.5839,
 'longitude': 143.0588,
 'surface temperature celcius': 99}
{'confidence': 100,
 'latitude': -36.5854,
 'longitude': 143.0462,
 'surface temperature celcius': 94}
{'confidence': 95,
 'latitude': -36.3014,
 'longitude': 143.203,
 'surface temperature celcius': 76}
{'confidence': 100,
 'latitude': -38.1917,
 'longitude': 147.0745,
 'surface temperature celcius': 94}
{'confidence': 95,
 'latitude': -38.1987,
 'longitude': 147.0666,
 'surface temperature celcius': 76}
{'confidence': 100,
 'latitude': -36.5454,
 'longitude': 144.7402,
 'surface temperature celcius': 96}
{'confidence': 92,
 'latitude': -36.7593,
 'longitude': 142.895,
 'surface temperature celcius': 69}
{'confidence': 90,
 'latitude': -36.3435,
 'longitude': 141.5513,
 'surface temperature celcius': 67}
{'confidence': 93,
 'latitude': -36.6942,
 '

In [11]:
print("query c")
#Find date, surface temperature (°C), air temperature (°C), relative humidity 
#and max wind speed on 15th and 16th of December 2018
results = collection.aggregate([
  {'$unwind': "$hotspot_histories"}, 
  {'$match': {'$or':[{"date": "2018-12-15"},{"date": "2018-12-16"}  ]}  } ,
  {
    "$project": {  
          "_id":0,            
          "date" : "$date",
          #"surface temperature celcius" :  { '$push': "$hotspot_histories.surface_temperature_celcius" },
          "surface temperature " : "$hotspot_histories.surface_temperature_celcius",
          "air temperature" : "$air_temperature_celcius",
          "relative humidity":"$relative_humidity",
          "max_wind_speed": "$max_wind_speed"
      }
  }
])

for result in results:
    pprint(result)


query c
{'air temperature': 18,
 'date': '2018-12-15',
 'max_wind_speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature ': 42}
{'air temperature': 18,
 'date': '2018-12-15',
 'max_wind_speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature ': 36}
{'air temperature': 18,
 'date': '2018-12-15',
 'max_wind_speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature ': 38}
{'air temperature': 18,
 'date': '2018-12-15',
 'max_wind_speed': 14.0,
 'relative humidity': 52.0,
 'surface temperature ': 40}
{'air temperature': 18,
 'date': '2018-12-16',
 'max_wind_speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature ': 43}
{'air temperature': 18,
 'date': '2018-12-16',
 'max_wind_speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature ': 33}
{'air temperature': 18,
 'date': '2018-12-16',
 'max_wind_speed': 13.0,
 'relative humidity': 53.7,
 'surface temperature ': 54}
{'air temperature': 18,
 'date': '2018-12-16',
 'max_wind_speed': 13.0,
 'relative humidi

In [10]:
print("query d")
# Find datetime, air temperature (°C), surface temperature (°C) and confidence 
#when the confidence is between 80 and 100

results = collection.aggregate([
  {'$unwind': "$hotspot_histories"}, 
  {'$match': { '$and':[ {"hotspot_histories.confidence": {"$gte":80}},
                       {"hotspot_histories.confidence": {"$lte":100}}  ] }} ,
  {
    "$project": {  
          "_id":0,            
          "datetime" : "$hotspot_histories.datetime",
          "air templerature" : "$air_temperature_celcius",
          "surface temperature celcius" : "$hotspot_histories.surface_temperature_celcius",
          "confidence" : "$hotspot_histories.confidence"
      }
  }
  #{'$group': { "_id": '$hotspot_histories.datetime','celcius':"$hotspot_histories.surface_temperature_celcius" }}
  #{'$group': { "_id": '$hotspot_histories.datetime','celcius': { '$push': "$hotspot_histories.surface_temperature_celcius" }}}
])


for result in results:
    #result=result.find({},{'surface_temperature_celcius':1})
    pprint(result)

query d
{'air templerature': 20,
 'confidence': 87,
 'datetime': '2018-03-06T05:06:30',
 'surface temperature celcius': 62}
{'air templerature': 20,
 'confidence': 85,
 'datetime': '2018-03-06T05:06:20',
 'surface temperature celcius': 59}
{'air templerature': 19,
 'confidence': 88,
 'datetime': '2018-03-07T04:16:10',
 'surface temperature celcius': 64}
{'air templerature': 23,
 'confidence': 86,
 'datetime': '2018-03-09T13:23:40',
 'surface temperature celcius': 41}
{'air templerature': 19,
 'confidence': 100,
 'datetime': '2018-03-10T04:48:40',
 'surface temperature celcius': 105}
{'air templerature': 19,
 'confidence': 100,
 'datetime': '2018-03-10T04:46:20',
 'surface temperature celcius': 109}
{'air templerature': 19,
 'confidence': 94,
 'datetime': '2018-03-10T04:45:30',
 'surface temperature celcius': 73}
{'air templerature': 19,
 'confidence': 97,
 'datetime': '2018-03-10T04:45:30',
 'surface temperature celcius': 80}
{'air templerature': 19,
 'confidence': 81,
 'datetime': '20

 'surface temperature celcius': 59}
{'air templerature': 16,
 'confidence': 83,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 56}
{'air templerature': 16,
 'confidence': 83,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 57}
{'air templerature': 16,
 'confidence': 91,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 68}
{'air templerature': 16,
 'confidence': 85,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 59}
{'air templerature': 16,
 'confidence': 81,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 54}
{'air templerature': 16,
 'confidence': 86,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 61}
{'air templerature': 16,
 'confidence': 84,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 80}
{'air templerature': 16,
 'confidence': 80,
 'datetime': '2018-04-13T04:26:30',
 'surface temperature celcius': 53}
{'air templerature': 16,
 'confidenc

 'datetime': '2018-05-03T04:02:20',
 'surface temperature celcius': 63}
{'air templerature': 10,
 'confidence': 84,
 'datetime': '2018-05-03T04:02:20',
 'surface temperature celcius': 58}
{'air templerature': 10,
 'confidence': 99,
 'datetime': '2018-05-03T04:02:20',
 'surface temperature celcius': 84}
{'air templerature': 10,
 'confidence': 95,
 'datetime': '2018-05-03T04:02:20',
 'surface temperature celcius': 76}
{'air templerature': 10,
 'confidence': 92,
 'datetime': '2018-05-04T04:53:10',
 'surface temperature celcius': 71}
{'air templerature': 10,
 'confidence': 80,
 'datetime': '2018-05-04T04:51:40',
 'surface temperature celcius': 54}
{'air templerature': 10,
 'confidence': 80,
 'datetime': '2018-05-04T04:51:20',
 'surface temperature celcius': 53}
{'air templerature': 10,
 'confidence': 94,
 'datetime': '2018-05-04T04:48:50',
 'surface temperature celcius': 73}
{'air templerature': 10,
 'confidence': 95,
 'datetime': '2018-05-04T04:48:20',
 'surface temperature celcius': 76}


In [12]:
#e. Find the top 10 records with the highest surface temperature (°C).

results = collection.aggregate([
  {
    "$project": {  
          "_id":0,            
          "hotspot_histories.datetime" : 1,
          "hotspot_histories.surface_temperature_celcius" : 1
          #"air temperature" : "$air_temperature_celcius",
          #"relative humidity":"$relative_humidity",
          #"max_wind_speed": "$max_wind_speed"
      }
  },
  {'$unwind': "$hotspot_histories"}, 
  {'$sort':{"hotspot_histories.surface_temperature_celcius":-1}},
  {'$limit':10}


])

for result in results:
    pprint(result)
    



{'hotspot_histories': {'datetime': '2018-04-18T04:52:00',
                       'surface_temperature_celcius': 124}}
{'hotspot_histories': {'datetime': '2018-04-04T04:32:50',
                       'surface_temperature_celcius': 123}}
{'hotspot_histories': {'datetime': '2018-05-01T04:14:20',
                       'surface_temperature_celcius': 122}}
{'hotspot_histories': {'datetime': '2018-03-18T03:50:50',
                       'surface_temperature_celcius': 121}}
{'hotspot_histories': {'datetime': '2018-04-04T04:32:40',
                       'surface_temperature_celcius': 120}}
{'hotspot_histories': {'datetime': '2018-04-13T04:28:10',
                       'surface_temperature_celcius': 120}}
{'hotspot_histories': {'datetime': '2018-05-13T04:40:20',
                       'surface_temperature_celcius': 120}}
{'hotspot_histories': {'datetime': '2018-04-18T04:44:50',
                       'surface_temperature_celcius': 120}}
{'hotspot_histories': {'datetime': '2018-04-04T04:40:00'

In [13]:
#f. Find the number of fire in each day. 
#You are required to only display the total number of fire and the date in the output.


results = collection.aggregate([
  { "$project": {
        "date": 1,
        "hotspot_histories": {
            "$cond": [
                { "$eq": [ "$hotspot_histories", [] ] },
                { "$const": [False] },
                "$hotspot_histories"
            ]
        }
    }},
  {'$unwind': "$hotspot_histories"}, 
  {'$group' : {  '_id': "$date",'total number': { '$sum': 
                                                {"$cond": [
                                                "$hotspot_histories",
                                                1,
                                                0
                                                           ]}
                                                  }}}


])

for result in results:
    pprint(result)

{'_id': '2018-01-01', 'total number': 0}
{'_id': '2018-12-31', 'total number': 0}
{'_id': '2018-12-29', 'total number': 0}
{'_id': '2018-12-28', 'total number': 0}
{'_id': '2018-12-26', 'total number': 0}
{'_id': '2018-12-24', 'total number': 1}
{'_id': '2018-12-23', 'total number': 0}
{'_id': '2018-12-20', 'total number': 0}
{'_id': '2018-12-19', 'total number': 0}
{'_id': '2018-12-16', 'total number': 15}
{'_id': '2018-12-13', 'total number': 1}
{'_id': '2018-12-10', 'total number': 2}
{'_id': '2018-12-09', 'total number': 4}
{'_id': '2018-12-07', 'total number': 0}
{'_id': '2018-12-04', 'total number': 0}
{'_id': '2018-12-03', 'total number': 0}
{'_id': '2018-12-02', 'total number': 0}
{'_id': '2018-12-01', 'total number': 0}
{'_id': '2018-11-29', 'total number': 8}
{'_id': '2018-12-22', 'total number': 0}
{'_id': '2018-11-26', 'total number': 0}
{'_id': '2018-11-19', 'total number': 0}
{'_id': '2018-11-12', 'total number': 5}
{'_id': '2018-11-07', 'total number': 0}
{'_id': '2018-1

In [15]:
#g. Find the average surface temperature (°C) for each day. 
#You are required to only display average surface temperature (°C) and the date in the output.

results = collection.aggregate([
  { "$project": {
        "date": 1,
        "hotspot_histories": {
            "$cond": [
                { "$eq": [ "$hotspot_histories", [] ] },
                { "$const": [False] },
                "$hotspot_histories"
            ]
        }
    }},
  {'$unwind': "$hotspot_histories"}, 
  {'$group' : {  '_id': "$date",'average surface temperature': { '$avg': 
                                                {"$cond": [
                                                "$hotspot_histories",
                                                "$hotspot_histories.surface_temperature_celcius",
                                                0
                                                           ]}
                                                  }}}


])

for result in results:
    pprint(result)


{'_id': '2018-01-01', 'average surface temperature': 0.0}
{'_id': '2018-12-31', 'average surface temperature': 0.0}
{'_id': '2018-12-29', 'average surface temperature': 0.0}
{'_id': '2018-12-28', 'average surface temperature': 0.0}
{'_id': '2018-12-26', 'average surface temperature': 0.0}
{'_id': '2018-12-24', 'average surface temperature': 32.0}
{'_id': '2018-12-23', 'average surface temperature': 0.0}
{'_id': '2018-12-20', 'average surface temperature': 0.0}
{'_id': '2018-12-19', 'average surface temperature': 0.0}
{'_id': '2018-12-16', 'average surface temperature': 57.8}
{'_id': '2018-12-13', 'average surface temperature': 60.0}
{'_id': '2018-12-10', 'average surface temperature': 46.0}
{'_id': '2018-12-09', 'average surface temperature': 58.25}
{'_id': '2018-12-07', 'average surface temperature': 0.0}
{'_id': '2018-12-04', 'average surface temperature': 0.0}
{'_id': '2018-12-03', 'average surface temperature': 0.0}
{'_id': '2018-12-02', 'average surface temperature': 0.0}
{'_id': 